# Setup

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import pytest as pt

from datetime import datetime

import car_cost_calculator as ccc

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [4]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

# Assumptions

In [ ]:
cpi = 0.02
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_efficiency_litres_per_100km = 7.0

purchase_price = 35000
age_at_purchase = 0
years_to_model = 15